In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

In [8]:
import csv
with open('Papers.csv', 'rb') as f:
    data = [row for row in csv.reader(f.read().splitlines())]

In [11]:
#Look at full data
limit = 3
l = 0
for d in data:
    print d
    l += 1
    if l > limit: break

['Id', 'Title', 'EventType', 'PdfName', 'Abstract', 'PaperText']
['5677', 'Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing', 'Poster', '5677-double-or-nothing-multiplicative-incentive-mechanisms-for-crowdsourcing.pdf', 'Crowdsourcing has gained immense popularity in machine learning applications for obtaining large amounts of labeled data. Crowdsourcing is cheap and fast, but suffers from the problem of low-quality data. To address this fundamental challenge in crowdsourcing, we propose a simple payment mechanism to incentivize workers to answer only the questions that they are sure of and skip the rest. We show that surprisingly, under a mild and natural no-free-lunch requirement, this mechanism is the one and only incentive-compatible payment mechanism possible. We also show that among all possible incentive-compatible  mechanisms (that may or may not satisfy no-free-lunch), our mechanism makes the smallest possible payment to spammers.  Interestingly, this u

In [14]:
#Look at one column of the data
limit = 3
l = 0
for d in data:
    print d[1] #title
    l += 1
    if l > limit: break

Title
Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing
Learning with Symmetric Label Noise: The Importance of Being Unhinged
Algorithmic Stability and Uniform Generalization


In [30]:
#LOAD ONE COLUMN OF THE DATA INTO A DOCUMENTS ARRAY
documents = []

limit = 1000
l = 0
for d in data:
    documents.append(d[1]) #title
    l += 1
    if l > limit: break

documents = documents[1:]
print documents

['Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing', 'Learning with Symmetric Label Noise: The Importance of Being Unhinged', 'Algorithmic Stability and Uniform Generalization', 'Adaptive Low-Complexity Sequential Inference for Dirichlet Process Mixture Models', 'Covariance-Controlled Adaptive Langevin Thermostat for Large-Scale Bayesian Sampling', 'Robust Portfolio Optimization', 'Logarithmic Time Online Multiclass prediction', 'Planar Ultrametrics for Image Segmentation', 'Expressing an Image Stream with a Sequence of Natural Sentences', 'Parallel Correlation Clustering on Big Graphs', 'Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks', 'Space-Time Local Embeddings', 'A Convergent Gradient Descent Algorithm for Rank Minimization and Semidefinite Programming from Random Linear Measurements', 'Smooth Interactive Submodular Set Cover', 'Galileo: Perceiving Physical Object Properties by Integrating a Physics Engine with Deep Learning'

In [31]:
#REMOVE PUNCTUATION AND COMMON WORDS WITHOUT DISCRIMINATORY QUALITIES ("STOP WORDS")
import string
exclude = set(string.punctuation)
stoplist = set('for a of the and to in or on from using via with by as are'.split())
#texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
texts = [[''.join(ch for ch in word if ch not in exclude) for word in document.lower().split() if word not in stoplist] for document in documents]
print texts

[['double', 'nothing', 'multiplicative', 'incentive', 'mechanisms', 'crowdsourcing'], ['learning', 'symmetric', 'label', 'noise', 'importance', 'being', 'unhinged'], ['algorithmic', 'stability', 'uniform', 'generalization'], ['adaptive', 'lowcomplexity', 'sequential', 'inference', 'dirichlet', 'process', 'mixture', 'models'], ['covariancecontrolled', 'adaptive', 'langevin', 'thermostat', 'largescale', 'bayesian', 'sampling'], ['robust', 'portfolio', 'optimization'], ['logarithmic', 'time', 'online', 'multiclass', 'prediction'], ['planar', 'ultrametrics', 'image', 'segmentation'], ['expressing', 'an', 'image', 'stream', 'sequence', 'natural', 'sentences'], ['parallel', 'correlation', 'clustering', 'big', 'graphs'], ['faster', 'rcnn', 'towards', 'realtime', 'object', 'detection', 'region', 'proposal', 'networks'], ['spacetime', 'local', 'embeddings'], ['convergent', 'gradient', 'descent', 'algorithm', 'rank', 'minimization', 'semidefinite', 'programming', 'random', 'linear', 'measurement

In [33]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint
pprint(texts)

[[],
 ['learning', 'symmetric', 'label', 'noise'],
 ['algorithmic', 'generalization'],
 ['adaptive', 'sequential', 'inference', 'process', 'mixture', 'models'],
 ['adaptive', 'langevin', 'largescale', 'bayesian', 'sampling'],
 ['robust', 'optimization'],
 ['logarithmic', 'time', 'online', 'multiclass', 'prediction'],
 ['image', 'segmentation'],
 ['an', 'image', 'sequence', 'natural'],
 ['parallel', 'clustering', 'graphs'],
 ['faster', 'towards', 'object', 'detection', 'region', 'networks'],
 ['local', 'embeddings'],
 ['gradient',
  'descent',
  'algorithm',
  'rank',
  'minimization',
  'semidefinite',
  'programming',
  'random',
  'linear',
  'measurements'],
 ['smooth', 'interactive', 'submodular', 'set', 'cover'],
 ['object', 'properties', 'integrating', 'deep', 'learning'],
 ['nearly', 'optimal'],
 ['neural', 'population', 'hierarchical', 'dynamics', 'models'],
 ['bayesian',
  'manifold',
  'learning',
  'locally',
  'linear',
  'latent',
  'variable',
  'model'],
 ['learning'],
 

In [34]:
#ASSOCIATE EVERY WORD WITH AN INTEGER (FOR COMPUTATIONAL EFFICIENCY)
dictionary = corpora.Dictionary(texts)
pprint(dictionary.token2id)

{u'3d': 127,
 u'accelerated': 112,
 u'accurate': 68,
 u'active': 183,
 u'adaptive': 11,
 u'adversarial': 260,
 u'algorithm': 47,
 u'algorithmic': 4,
 u'algorithms': 90,
 u'an': 25,
 u'analysis': 171,
 u'answering': 321,
 u'application': 293,
 u'applications': 233,
 u'approach': 109,
 u'approval': 317,
 u'approximate': 279,
 u'approximation': 292,
 u'arbitrary': 210,
 u'asynchronous': 289,
 u'augmented': 349,
 u'automated': 350,
 u'backpropagation': 243,
 u'bandits': 75,
 u'based': 163,
 u'bayesian': 12,
 u'belief': 266,
 u'beyond': 160,
 u'bidirectional': 83,
 u'bipartite': 194,
 u'blackbox': 174,
 u'block': 178,
 u'bounding': 86,
 u'bounds': 271,
 u'brain': 170,
 u'carlo': 222,
 u'causal': 274,
 u'chain': 185,
 u'class': 126,
 u'classification': 169,
 u'classifiers': 275,
 u'clustering': 28,
 u'combinatorial': 283,
 u'completion': 246,
 u'complex': 340,
 u'complexity': 252,
 u'compressive': 150,
 u'connections': 220,
 u'consistency': 335,
 u'consistent': 312,
 u'constrained': 128,
 u'

In [35]:
#REPRESENT EVERY DOCUMENT AS A SPARSE ARRAY OF WORD IDS AND COUNTS (FOR COMPUTATIONAL EFFICIENCY)
corpus = [dictionary.doc2bow(text) for text in texts]
pprint(corpus)

[[],
 [(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)],
 [(11, 1), (12, 1), (13, 1), (14, 1), (15, 1)],
 [(16, 1), (17, 1)],
 [(18, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(23, 1), (24, 1)],
 [(23, 1), (25, 1), (26, 1), (27, 1)],
 [(28, 1), (29, 1), (30, 1)],
 [(31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)],
 [(37, 1), (38, 1)],
 [(39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1)],
 [(49, 1), (50, 1), (51, 1), (52, 1), (53, 1)],
 [(2, 1), (34, 1), (54, 1), (55, 1), (56, 1)],
 [(57, 1), (58, 1)],
 [(9, 1), (59, 1), (60, 1), (61, 1), (62, 1)],
 [(2, 1), (12, 1), (48, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)],
 [(2, 1)],
 [(7, 1), (9, 1), (68, 1), (69, 1)],
 [(17, 1), (70, 1), (71, 1)],
 [(4, 1), (72, 1)],
 [],
 [(73, 1), (74, 1), (75, 1)],
 [(76, 1), (77, 1), (78, 1), (79, 2)],
 [(80, 1), (81, 1), (82, 1)],
 [(36, 1), (72, 1), (83, 1), (84, 1), (85, 1)],
 [(86

In [39]:
#COMPUTE AND APPLY MAPPING FROM COUNTS TO NUMBERS THAT ACCOUNT FOR DOCUMENT FREQUENCY
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[]
[(0, 0.5683469488014847), (1, 0.5683469488014847), (2, 0.17591258443933855), (3, 0.5683469488014847)]
[(4, 0.7546821231853805), (5, 0.6560906133640428)]
[(6, 0.49702915811192194), (7, 0.2953355908175191), (8, 0.43672849921975226), (9, 0.26330564615147284), (10, 0.49702915811192194), (11, 0.39829546249435677)]
[(11, 0.3966236981061267), (12, 0.34619164933573066), (13, 0.47099936297314193), (14, 0.5693186439367763), (15, 0.42056731420274585)]
[(16, 0.801178923907056), (17, 0.598424875725543)]
[(18, 0.44038186537774215), (19, 0.48282350182877126), (20, 0.35944902186593214), (21, 0.5553779585652389), (22, 0.3678274086412745)]
[(23, 0.6129705569714845), (24, 0.7901057500651846)]
[(23, 0.4223625752357862), (25, 0.5124552955270989), (26, 0.5444161966847262), (27, 0.5124552955270989)]
[(28, 0.6006350597610682), (29, 0.5277073525843999), (30, 0.6006350597610682)]
[(31, 0.46954919580744725), (32, 0.46954919580744725), (33, 0.46954919580744725), (34, 0.38845973948890844), (35, 0.37232473407929

In [46]:
#PRODUCE LSI MODELS FOR TOPIC NUMBERS FROM 2 TO 10
for N in range(2,11):
    print N, 'topics:'
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=N) 
    corpus_lsi = lsi[corpus_tfidf]
    pprint(lsi.print_topics(N))
    print

2 topics:
[(0,
  u'0.580*"learning" + 0.423*"networks" + 0.274*"neural" + 0.231*"deep" + 0.180*"convolutional" + 0.166*"optimization" + 0.161*"models" + 0.143*"recurrent" + 0.133*"inference" + 0.124*"bayesian"'),
 (1,
  u'-0.539*"networks" + 0.482*"learning" + -0.381*"neural" + -0.249*"convolutional" + 0.232*"inference" + 0.156*"variational" + -0.130*"deep" + 0.123*"online" + -0.104*"recurrent" + 0.081*"stochastic"')]

3 topics:
[(0,
  u'0.581*"learning" + 0.423*"networks" + 0.275*"neural" + 0.231*"deep" + 0.180*"convolutional" + 0.166*"optimization" + 0.161*"models" + 0.143*"recurrent" + 0.132*"inference" + 0.124*"bayesian"'),
 (1,
  u'-0.538*"networks" + 0.482*"learning" + -0.383*"neural" + -0.248*"convolutional" + 0.230*"inference" + 0.156*"variational" + -0.130*"deep" + 0.125*"online" + -0.105*"recurrent" + 0.081*"data"'),
 (2,
  u'0.517*"inference" + -0.463*"learning" + 0.421*"variational" + 0.276*"models" + 0.199*"optimization" + 0.137*"probabilistic" + 0.125*"stochastic" + 0.103

In [49]:
#PRODUCE LDA MODELS FOR TOPIC NUMBERS FROM 2 TO 10
for N in range(2,11):
    print N, 'topics:'
    lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=N,passes=10,alpha='auto')
    pprint(lda.print_topics(N))
    print

2 topics:
[(0,
  u'0.034*models + 0.026*inference + 0.023*learning + 0.016*fast + 0.013*estimation + 0.013*data + 0.013*efficient + 0.012*variational + 0.012*optimal + 0.011*algorithms'),
 (1,
  u'0.054*learning + 0.032*networks + 0.026*optimization + 0.019*stochastic + 0.019*neural + 0.016*deep + 0.013*robust + 0.012*convolutional + 0.012*online + 0.012*bandits')]

3 topics:
[(0,
  u'0.032*learning + 0.029*stochastic + 0.021*analysis + 0.019*optimization + 0.017*spectral + 0.013*model + 0.013*convergence + 0.013*gradient + 0.012*adaptive + 0.012*gaussian'),
 (1,
  u'0.060*learning + 0.022*efficient + 0.018*data + 0.017*robust + 0.015*models + 0.014*bayesian + 0.014*local + 0.013*optimization + 0.012*submodular + 0.012*deep'),
 (2,
  u'0.040*networks + 0.030*learning + 0.026*inference + 0.025*models + 0.022*neural + 0.016*deep + 0.015*optimization + 0.015*prediction + 0.014*bounds + 0.013*optimal')]

4 topics:
[(0,
  u'0.051*networks + 0.032*neural + 0.027*optimization + 0.026*inferenc